In [1]:
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import time
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 32
EPOCH = 8
LR = 0.0001
new_model_train = True
device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()
model_type = "PreTrained_ResNet50_SavemodelTest"
dataset = "dataset"
save_model_path = f"result/{model_type}{dataset}.pt"
save_history_path = f"result/{model_type}history{dataset}.pt"

In [3]:
device

'cpu'

In [4]:
train_dir = 'dataset/train'
valid_dir = 'dataset/valid'
test_dir = 'dataset/test'

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_DS = datasets.ImageFolder(train_dir, transform=train_transform)
valid_DS = datasets.ImageFolder(valid_dir, transform=valid_transform)
test_DS = datasets.ImageFolder(test_dir, transform=test_transform)

train_DL = DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
valid_DL = DataLoader(valid_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class ResNet(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024, momentum=0.05),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512, momentum=0.05),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        return x

In [ ]:
def loss_epoch(model, DL, criterion, optimizer = None):
    N = len(DL.dataset)
    rloss = 0; rcorrect = 0
    for x_batch, y_batch in tqdm(DL, leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        y_hat = model(x_batch)
        loss = criterion(y_hat, y_batch)
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_b = loss.item() * x_batch.shape[0]
        rloss += loss_b
        pred = torch.argmax(y_hat, dim=1)
        corrects_b = torch.sum(pred == y_batch).item()
        rcorrect += corrects_b
    loss_e = rloss/N
    accruracy_e = rcorrect/N * 100
    return loss_e, accruracy_e, rcorrect